<a href="https://colab.research.google.com/github/SebastianLealT/OPTIMIZACION-1/blob/main/Optimizaci%C3%B3n_con_restricciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Problema de optimización con restricciones
###Johan Sebastian Leal
###Curso de Optimización I
###Universidad del valle, Cali

En este cuaderno nos centraremos en resolver dos casos de optimización con restricciones no lineales para una misma función objetivo. La función escogida es conocida como *Booth Function* que estará definida en la primera sección de este documento.

Para la realización de este proyecto será necesario importar los siguientes paquetes:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

##Función a evaluar y algunos otros cálculos

Definimos nuestra función objetivo, en este caso será:

$$f(x,y)=(x+2y-7)^{2}+(2x+y-5)^{2}$$

la cual es conocida como *Booth Function*.

In [2]:
def booth(x):
  return ((x[0] + (2*x[1]) -7)**2) + (((2*x[0]) + x[1] - 5)**2)

Habiendo definido la función objetivo, será necesario conocer el Jacobiano y la Hessiana de la función en cuestión, para ello realizamos algunos cálculos manuales y definimos cada una de estas operaciones como se muestra:

1. Jacobiano de $f(x,y)$:
$$J(x,y)=\begin{bmatrix} 2(x+2y-7)+4(2x+y-5)\\ 4(x+2y-7)+2(2x+y-5)\end{bmatrix}$$

2. Hessiana de $f(x,y)$:
$$H(x,y)=\begin{bmatrix} 10& 8\\ 8& 10\end{bmatrix}$$

In [3]:
#Se define el jacobiano
def Jf(x):
  der = np.zeros_like(x)
  der[0]= (2*(x[0] + (2*x[1]) -7)) + (4*((2*x[0]) + x[1] - 5))
  der[1]= (4*(x[0] + (2*x[1]) -7)) + (2*((2*x[0]) + x[1] - 5))
  return der

#Se define la Hessiana
def Hf(x):
  x = np.asarray(x)
  H = np.diag([10 , 10 ])
  H[1,0]= 8
  H[0,1]= 8
  return H

##Problema de optimización \#1

Minimice la función $f(x,y)$ sujeto a las siguientes restricciones:

Restricciones lineales:

$$x+2y \leq 1$$
$$ 2x+y= 1 $$

Restricciones no lineales:
$$x^2 + y \leq 1$$
$$x^2 - y \leq 1 $$

Cotas:
$$ -10 \leq x \leq 10 $$
$$ -10 \leq y \leq 10$$

In [4]:
#[cotas inferiores, cotas superiores]
bounds = optimize.Bounds([-10, -10], [10,10])
bounds

Bounds([-10, -10], [10, 10])

##Problema de optimización \#2

Minimice la función $f(x,y)$ sujeto a las siguientes restricciones:

Restricciones lineales:

$$x+2y \leq 1$$
$$ 2x+y= 1 $$

Restricciones no lineales:
$$x^2 + y \leq 1$$
$$x^2 - y \leq 1 $$

Cotas:
$$ -10 \leq x \leq 10 $$
$$ -10 \leq y \leq 10$$

##Conclusiones

##Referencias

1. Heath, M. T. (2018). Scientific Computing: An Introductory Survey, Revised Second Edition. Society for Industrial and Applied Mathematics.
2. Chong, E. K., & Zak, S. H. (2004). An introduction to optimization. John Wiley & Sons.